In [8]:
import numpy as np
import pandas as pd
from numpy.linalg import inv
from scipy import sparse

In [9]:
def prepare_rating_matrix():
    ratings = pd.read_csv('Purchases.csv')
    
    # of customers
    m = ratings['Customer_ID'].max() + 1
    
    # of products
    n = ratings['Product_ID'].max() + 1
    
    #matrix of ratings
    m1 = np.zeros((m,n))
    
    i = ratings[:]['Customer_ID']
    j = ratings[:]['Product_ID']
    
    m1[i, j] = 1
       
    return m1

In [10]:
def prepare_feature_matrix():
    customer_features = pd.read_csv('Customer_Features.csv')
    # convert into numpy array of m rows, n columns
    a = customer_features.as_matrix()
    
    product_features = pd.read_csv('Product_Features.csv')
    b = product_features.as_matrix()
    
    return a[:, 1:], b[:, 1:]

In [11]:
A, B = prepare_feature_matrix()
m1 = prepare_rating_matrix()
m1.shape

(301, 284)

In [12]:
#next steps
# efficient way of doing test-train split using np.axis
#train the model
#efficient way of doing eval - AUC
#run model and get result

#set up langragian formulation - splitting m,n
#set up RDD datapreparation, training and eval
   #data preparation - map actual id to serial id 
   #efficient test-train split

#run model on databricks and get result
#run model on AWS and compare running time for different values of k = # of nodes
#document and upload on Github and we are done!!!!

#https://stackoverflow.com/questiohttps://stackoverflow.com/questions/13272453/multiplying-numpy-scipy-sparse-and-dense-matrices-efficientlyns/13272453/multiplying-numpy-scipy-sparse-and-dense-matrices-efficiently

In [47]:
#utility to flag certain purchases for testing, prepare the weighting matrix
def remove_purchases(vector, fraction):
    d = len(vector) // 2
    tmp_m1  = vector[:d]
    C = vector[d:]
    s = np.sum(tmp_m1)
    sample = int(s*fraction)
    if ((s > 0) & (sample > 0)):
        shape = tmp_m1.shape
        p = np.full(shape,1.0)
        p[tmp_m1 == 0] = 0
        valid = tmp_m1[tmp_m1 > 0].size
        p = p/valid
        inds = np.random.choice(tmp_m1.size, sample, replace = False, p = p)
        C[inds] = -2
    return np.concatenate((tmp_m1, C))        

In [48]:
def prepare_weighting_matrix(m1, alpha = 40, fraction = .2): 
    #alpha adjusts weighting between purchases, non-purchases
    #fraction determines percent of purchases for each customer that needs to be set aside for testing and not used during training
    C = m1*alpha + 1
    x = np.hstack((m1, C))
    mat =  np.apply_along_axis(lambda vector: remove_purchases(vector, fraction), 1, x)
    d = mat.shape[1] // 2
    return (mat[:, :d], mat[:, d:])

In [49]:
m1, C = prepare_weighting_matrix(m1)
C.shape

(301, 284)

In [50]:
def initialize_params(m1, C, A, B, f):
    m = m1.shape[0] #get rows
    n = m1.shape[1] #get columns
    
    assert A.shape[0] ==   m, "customer feature dimension incompatibility"
    assert B.shape[0] == n,  "product feature dimension incompatibility"
    
    k = A.shape[1]
    l = B.shape[1]
    
    #initialize trainable parameters
    X = np.random.randn(m, f) #m*f matrix for user latent factors
    Y = np.random.randn(n, f) #n*f matrix for item latent factors
    D = np.random.randn(l, f) #embedding to transform item features into the f-dimensional factor space
    G = np.random.randn(k, f) #embedding to transform user features into the f-dimensional factor space
    d = np.random.randn(l) #embedding to transform item features into the f-dimensional factor space
    g = np.random.randn(k) #embedding to transform user features into the f-dimensional factor space
    Bias_user = np.random.randn(m) #bias for m users
    Bias_item = np.random.randn(n) #bias for n users
    
    return X, Y, D, G, d, g, Bias_user, Bias_item
    

In [51]:
X, Y, D, G, d, g, Bias_user, Bias_item = initialize_params(m1, C, A, B, 10)

In [52]:
def update_user_factors(m1, C, A, B, L, X, Y, D, G, d, g, Bias_user, Bias_item, f):
    B_ = np.matmul(B, D)
    for u in range(0, m1.shape[0]):
            Filter = C[u,:]            #apply the filter to the inputs before updating the customer's parameters
            p_u = m1[u, :]
            p_u = p_u[Filter > -1]            
            alpha_u = A[u,:]                      
            term1 = np.matmul(Y + B_, np.matmul(alpha_u, G))
            term1 = term1[Filter > -1]            
            term2 = np.matmul(B, d)
            term2 = term2[Filter > -1]            
            term3 = np.matmul(g, alpha_u)            
            term4 = Bias_item[Filter > -1]            
            p_u = p_u  - term1 - term2  - term3 - term4
            
            n_adj = p_u.size
            temp = C[u, :]
            temp = temp[temp > -1]
            c_u = temp*np.eye(n_adj) 
            
            ones = np.ones((n_adj, 1))
            Yadj = Y + B_
            Yadj = Yadj[Filter > -1, :]            
            Yadj = np.concatenate((Yadj, ones), axis = 1)
            
            
            I_x = L*np.eye(f + 1)
                        
            #Implement equation 1
            step1 = np.matmul(Yadj.T, (np.matmul(c_u,Yadj))) + I_x
            step2 = inv(step1)
            updates = np.matmul(step2, np.matmul(Yadj.T, np.matmul(c_u, p_u)))
            #print (updates.shape)
            
            X[u,:] = updates[:f]
            Bias_user[u] = updates[f:]
            
            #print ('%s %d' % ('Updated factors for user #: ', u+1))            
    

In [53]:
def update_item_factors(m1, C, A, B, L, X, Y, D, G, d, g, Bias_user, Bias_item, f):
    A_ = np.matmul(A, G)
    for i in range(0, m1.shape[1]):
        Filter = C[:,i]            #apply the filter to the inputs before updating the item's parameters
        p_i = m1[:, i]
        p_i = p_i[Filter > -1]   
        beta_i = B[i,:]                      
        term1 = np.matmul(X + A_, np.matmul(beta_i, D))
        term1 = term1[Filter > -1]            
        term2 = np.matmul(A, g)
        term2 = term2[Filter > -1]            
        term3 = np.matmul(d, beta_i)            
        term4 = Bias_user[Filter > -1]            
        p_i = p_i  - term1 - term2  - term3 - term4
        
        n_adj = p_i.size
        temp = C[:, i]
        temp = temp[temp > -1]
        c_i = temp*np.eye(n_adj) 
        
        ones = np.ones((n_adj, 1))
        Xadj = X + A_
        Xadj = Xadj[Filter > -1, :]            
        Xadj = np.concatenate((Xadj, ones), axis = 1)
        
        I_y = L*np.eye(f + 1)
        
        #update user factors, user bias
        step1 = np.matmul(Xadj.T, (np.matmul(c_i,Xadj))) +I_y
        step2 = inv(step1)
        updates = np.matmul(step2, np.matmul(Xadj.T, np.matmul(c_i, p_i)))
        #print (updates.shape)
            
        Y[i,:] = updates[:f]
        Bias_item[i] = updates[f:]
            
        #print ('%s %d' % ('Updated factors for item #: ', i+1))    

In [54]:
def update_customer_embedding(m1, C, A, B, L_G, X, Y, D, G, d, g, Bias_user, Bias_item, f):
        
    m = m1.shape[0]
    n = m1.shape[1]
    k = A.shape[1]
    l = B.shape[1]
      
    B_ = np.matmul(B, D) + Y
    Y_ = np.tensordot(A, B_, axes = 0)   
    Y_ = np.swapaxes(Y_, 1, 2)    
    Y_ = np.reshape(Y_, (m, n, -1))
    
    
    X_ = np.matmul(X, B_.T)    
    Bu = np.expand_dims(Bias_user, axis = 1)
    Bi = np.expand_dims(Bias_item, axis = 0)
    
    m1_adj = m1  -  Bi -  np.expand_dims(np.matmul(B, d), axis = 0)  - Bu - X_
    
    A_ = np.zeros((m,n,k)) + np.expand_dims(A, axis = 1)
    Y_ = np.concatenate((Y_, A_), axis = 2)
    
    Y_ = np.reshape(Y_, (m*n, -1))
    C_ = np.squeeze(np.reshape(C, (m*n, -1)))
    m1_adj = np.squeeze(np.reshape(m1_adj, (m*n, -1)))
    Filter = C_
    

    C_adj = C_[Filter > -1]    
    n_adj = C_adj.size
    
    C_ui = sparse.csr_matrix.multiply(sparse.eye(n_adj), C_adj)
    p = m1_adj[Filter > -1]
    Y_ = Y_[Filter > -1, :]
   
    
    I_G = L_G*np.eye(k*f + k)
    
    step1 = np.matmul(Y_.T, C_ui.dot(Y_)) + I_G
    step2 = inv(step1)
    updates = np.matmul(step2, np.matmul(Y_.T, C_ui.dot(p)))
    
    G = np.reshape(updates[:k*f],(k,f))
    g = updates[-k:]       

In [55]:
def update_item_embedding(m1, C, A, B, L_D, X, Y, D, G, d, g, Bias_user, Bias_item, f):    
    
    m = m1.shape[0]
    n = m1.shape[1]
    k = A.shape[1]
    l = B.shape[1]
    
    A_ = np.matmul(A, G) + X
    X_ = np.tensordot(A_, B, axes = 0)
    X_ = np.swapaxes(X_, 1, 2) 
    X_ = np.swapaxes(X_, 2, 3) 
    X_ = np.reshape(X_, (m, n, -1))
    
    Y_ = np.matmul(A_, Y.T)    
    Bu = np.expand_dims(Bias_user, axis = 1)
    Bi = np.expand_dims(Bias_item, axis = 0)
    m1_adj = m1  -  Bi -  np.expand_dims(np.matmul(A, g), axis = 1)  - Bu - Y_
    
   
    B_ = np.zeros((m,n,l)) + np.expand_dims(B, axis = 0)
    X_ = np.concatenate((X_, B_), axis = 2)
    
    X_ = np.reshape(X_, (m*n, -1))
    C_ = np.squeeze(np.reshape(C, (m*n, -1)))
    m1_adj = np.squeeze(np.reshape(m1_adj, (m*n, -1)))
    Filter = C_
    
    C_adj = C_[Filter > -1]    
    n_adj = C_adj.size
    
    C_ui = sparse.csr_matrix.multiply(sparse.eye(n_adj), C_adj)
    p = m1_adj[Filter > -1]
    X_ = X_[Filter > -1, :]
    
    I_D = L_D*np.eye(l*f + l)
    
    step1 = np.matmul(X_.T, C_ui.dot(X_)) + I_D
    step2 = inv(step1)
    updates = np.matmul(step2, np.matmul(X_.T, C_ui.dot(p)))
    
    D = np.reshape(updates[:l*f],(l,f))
    d = updates[-l:]    

In [68]:
#core model training based on Alternating Least Squares (ALS)
# f is the number of latent factors to estimate per customer, product
#Sweeps specifies # of iterations until convergence
def modeltrain(m1, C, A, B, L_U = 2, L_I = 2, L_G = 2, L_D = 2, f = 20, sweeps = 10):    

    X, Y, D, G, d, g, Bias_user, Bias_item =  initialize_params(m1, C, A, B, f)
    #Each sweep comprises an update of user factors, item factors, user/item feature to factor embedding
    for itr in range(0, sweeps):
        print ('%s %d' % ('Starting iteration #: ', itr + 1))  
        update_user_factors(m1, C, A, B, L_U, X, Y, D, G, d, g, Bias_user, Bias_item, f)
        update_item_factors(m1, C, A, B, L_I, X, Y, D, G, d, g, Bias_user, Bias_item, f)
        update_customer_embedding(m1, C, A, B, L_G, X, Y, D, G, d, g, Bias_user, Bias_item, f)
        update_item_embedding(m1, C, A, B, L_D, X, Y, D, G, d, g, Bias_user, Bias_item, f)     
    return X, Y, D, G, d, g, Bias_user, Bias_item

In [69]:
X, Y, D, G, d, g, Bias_user, Bias_item = modeltrain(m1, C, A, B)

Starting iteration #:  1
Starting iteration #:  2
Starting iteration #:  3
Starting iteration #:  4
Starting iteration #:  5
Starting iteration #:  6
Starting iteration #:  7
Starting iteration #:  8
Starting iteration #:  9
Starting iteration #:  10


In [70]:
def predictions(X, Y, D, G, d, g, Bias_user, Bias_item, A, B): #returns m by n dense matrix of predictions
    temp1 = np.matmul((np.matmul(A,G) + X), (np.matmul(B,D) + Y).T)
    temp2 = np.expand_dims(np.matmul(A,g), axis = 1)
    temp3 = np.expand_dims(np.matmul(B,d), axis = 0)
    return temp1 + temp2 + temp3

In [71]:
preds = predictions(X, Y, D, G, d, g, Bias_user, Bias_item, A, B)
preds.shape

(301, 284)

In [72]:
def modeleval_AUC(m1, C, pred1):
    result = 0.0   
    m = m1.shape[0]   
    count = 0
    for i in range(0,m):
        num = 0
        pred2 = pred1[i,:]
        preds = pred2[(m1[i,:] == 0) & (C[i,:] > 0)]
        testscore = pred2[C[i,:] == -2]
        l = testscore.size
        if ((l > 0) & (preds.size > 0)):
            for j in range(0,l):
                temp = testscore[j] - preds
                num = num + sum(temp > 0)
            result = result + float(num)/float(preds.size*l)
            count = count + 1 
        
    return (result/count)

In [73]:
modeleval_AUC(m1, C, preds)

0.7965624443080479